In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import json
import torch
import numpy as np
import time
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import clear_output
from nerfstudio.utils.eval_utils import eval_setup
from plane_nerf.inerf_trainer import load_data_into_trainer
from plane_nerf.inerf_utils import load_eval_image_into_pipeline, inerf


/home/user/.local/lib/python3.10/site-packages/tinycudann/modules.py:31: UserWarning: System has multiple GPUs with different compute capabilities: [61, 52]. Using compute capability 52 for best compatibility. This may result in suboptimal performance.
  warnings.warn(


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
os.chdir('/workspace/plane-nerf')
MODEL_PATH = "/workspace/plane-nerf/outputs/jackal_floor_training_data_1/plane-nerf-fast/2024-03-13_105236"
DATA_PATH = "/workspace/plane-nerf/data/jackal_floor_evaluation_data"
GROUND_TRUTH_PATH = os.path.join(DATA_PATH, "ground_truth.json")
SAVE_FILE_NAME = "eval_results_"+str(time.strftime("%Y-%m-%d_%H%M%S"))+".csv"

config_path = os.path.join(MODEL_PATH, "config.yml")
config, pipeline, _, _ = eval_setup(
                        Path(config_path),
                        test_mode="inference",
                    )
transform_file_path = "transforms_10.json"
with open(os.path.join(DATA_PATH, transform_file_path)) as f:
    transform = json.load(f)

with open(GROUND_TRUTH_PATH) as f:
    ground_truth = json.load(f)



tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Falling back to CutlassMLP. For maximum performance, raise the target GPU architecture to 75+.
tiny-cuda-nn warning: FullyFusedMLP is not supported for the selected architecture 52. Fal

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/jackal_floor_training_data_1/plane-nerf-fast/2024-03-13_105236/nerfstudio_models/step-000009999.ckpt

In [4]:
output = []

for f in range(len(transform["frames"])):
    transform_dup = transform.copy()
    transform_dup["frames"] = [transform["frames"][f]]
        
    pipeline = load_eval_image_into_pipeline(pipeline,DATA_PATH,transform_dup)

    config.pipeline.datamanager.pixel_sampler.num_rays_per_batch = 4096 

    trainer = load_data_into_trainer(
        config,
        pipeline,
        plane_optimizer = True
    )
    trainer.pipeline.datamanager.KERNEL_SIZE = 5
    trainer.pipeline.datamanager.THRESHOLD = 40
    trainer.pipeline.datamanager.METHOD = "sift"

    clear_output(wait=True)
    print(f)

    tf = ground_truth["frames"][f]["transform_matrix"]
    tf = np.asarray(tf)
    tf = tf[:3, :4 ]
    ground_truth_poses = [tf]
    ground_truth_poses = torch.tensor(ground_truth_poses).to(pipeline.device)
    
    ans = inerf(trainer, ITERATION=1000, GROUND_TRUTH_POSE=ground_truth_poses)
    output.append(ans["store"].detach().cpu().numpy().flatten())
    np.savetxt(os.path.join(MODEL_PATH,SAVE_FILE_NAME), np.asarray(output), delimiter=",")

108
Number of keypoints:  105
Number of rays:  4479
Reduce the number of rays
Final number of rays:  4096


In [ ]:
print(ans["store"])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 print(ans["store"])                                                                          │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ans' is not defined